#  Modeling noise in $\alpha$

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
import pymc3 as pm
import sys
sys.path.insert(0, '../../')
import mwc.viz
import mwc.stats
import mwc.bayes
mwc.viz.personal_style()

$$
\langle (I_1 - I_2 )^2 \rangle = \alpha I_\text{tot}
$$


### Model I - No noise.

$$
I_{tot} = \alpha N_{tot}
$$



### Model II -  Normal measurement noise

$$
I_{tot} = \alpha N_{tot}  + \epsilon\, ;\, \epsilon \sim \mathcal{Normal}(0, \sigma)
$$


### Model III - Cellular noise

$$
I_{tot} = \sum\limits_i^N \alpha_i + \epsilon_i \,;\, \epsilon \sim \mathcal{Normal}(0, \sigma)
$$

### Model IV - Cellular and measurement noise

$$
I_{tot} = \gamma + \sum\limits_i^N \alpha_i + \epsilon_i\, ;\, \epsilon \sim \mathcal{Normal}(0, \sigma)\, ;\, \gamma \sim \mathcal{Normal}(0, \zeta)
$$


In [ ]:
# Set up some cells. 
n_div = 1000
